<a href="https://colab.research.google.com/github/CatalystM47/Deep_Learning/blob/main/ASL%20classification_epoch60.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Augmentation

In [39]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("sign_mnist_train.csv")
valid_df = pd.read_csv("sign_mnist_valid.csv")

In [40]:
y_train = train_df["label"]
y_valid = valid_df["label"]

In [41]:
del train_df["label"]
del valid_df["label"]

x_train = train_df.values
x_valid = valid_df.values

In [42]:
from PIL import Image

image = x_train[0,:].reshape(28,28)
img_pil = Image.fromarray(np.uint8(image))

In [43]:
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.Resize(32),
     transforms.RandomCrop(28)]
)

In [44]:
img_pil

In [45]:
transform(img_pil)

### 1. Data preparation

In [46]:
import torch
from torch.utils.data import Dataset, DataLoader

In [47]:
class ASL_Dataset(Dataset):
    def __init__(self, path, transform=None):
        super(ASL_Dataset, self).__init__()

        train_df = pd.read_csv(path)
        y_train = train_df["label"]
        del train_df["label"]
        x_train = train_df.values

        self.x_train = x_train   # Nx784
        self.y_train = y_train
        self.transform = transform

    def __getitem__(self, index):
        x_train_sample = np.uint8( self.x_train[index].reshape(28,28) )

        if self.transform:
            x_train_pil = Image.fromarray(x_train_sample)
            x_train_np = np.array(self.transform(x_train_pil))
            x_train_tensor = torch.tensor( x_train_np[np.newaxis,:,:] ).float() # 1x28x28
        else:
            x_train_tensor = torch.tensor( x_train_sample[np.newaxis,:,:] ).float()

        y_train_tensor = torch.tensor( self.y_train[index] )

        return x_train_tensor, y_train_tensor

    def __len__(self):
        return self.x_train.shape[0]

In [48]:
training_data = ASL_Dataset("sign_mnist_train.csv", transform=transform)
validation_data = ASL_Dataset("sign_mnist_valid.csv")

In [49]:
train_loader = DataLoader(dataset=training_data, batch_size=1000, shuffle=True)
validation_loader = DataLoader(dataset=validation_data, batch_size=1000, shuffle=False)

### 2. Constructing a convolutional neural network

In [50]:
import torch.nn as nn

In [51]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(32*7*7, 24)

    def forward(self, x):  ## 1x28x28
        x = self.conv1(x)   ## 16x28x28
        x = self.relu(x)
        x = self.pool(x)    ## 16x14x14

        x = self.conv2(x)   ## 32x14x14
        x = self.relu(x)
        x = self.pool(x)    ## 32x7x7

        x = self.conv3(x)   ## 32x7x7
        x = self.relu(x)

        x = x.view(-1, 32*7*7)
        x = self.fc(x)

        return x

In [52]:
convnet = ConvNet()

### 3. Loss function and Optimization method

In [53]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnet.parameters(), lr=0.001)

### 4. Training of a neural network 

In [57]:
for epoch in range(20):
    loss_val = 0
    for itr, data in enumerate(train_loader):
        optimizer.zero_grad()

        inputs, labels = data

        pred = convnet(inputs)
        loss = loss_function(pred, labels)

        loss.backward()
        optimizer.step()

        loss_val += loss.item()

    print("Epoch:", epoch+1, "  , Loss:", loss_val)

Epoch: 1   , Loss: 0.42198352236300707
Epoch: 2   , Loss: 0.36196892336010933
Epoch: 3   , Loss: 0.27660146448761225
Epoch: 4   , Loss: 0.2315736678428948
Epoch: 5   , Loss: 0.22454083431512117
Epoch: 6   , Loss: 0.18077853857539594
Epoch: 7   , Loss: 0.16299481177702546
Epoch: 8   , Loss: 0.2356542577035725
Epoch: 9   , Loss: 0.22914747428148985
Epoch: 10   , Loss: 0.1876771568786353
Epoch: 11   , Loss: 0.19568002340383828
Epoch: 12   , Loss: 0.1521437035407871
Epoch: 13   , Loss: 0.13424216769635677
Epoch: 14   , Loss: 0.10356573411263525
Epoch: 15   , Loss: 0.11988488119095564
Epoch: 16   , Loss: 0.11932707694359124
Epoch: 17   , Loss: 0.09261183568742126
Epoch: 18   , Loss: 0.10227902722544968
Epoch: 19   , Loss: 0.11461713362950832
Epoch: 20   , Loss: 0.07670171139761806


### 5. Prediction and evaludation for a test set

In [59]:
pred_list = []
label_list = []
for itr, data in enumerate(validation_loader):
    inputs, labels = data
    pred_test = convnet(inputs)
    pred_category = torch.argmax(pred_test, dim=1)

    pred_list = pred_list + list(pred_category)    
    label_list = label_list + list(labels)

accu = np.mean( np.array(pred_list)==np.array(label_list) )
print("Test accuracy:", accu)

Test accuracy: 0.9136921360847742
